In [1]:
import requests
import pytz

import json
from datetime import datetime, time as datetime_time, timedelta
import calendar


In [2]:
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r


In [3]:
def make_api_request(stop_id, bearer):
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/OperationMonitoring/4.0/PassingTimeByPoint/" + stop_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()

In [4]:
def parse_waiting_times(json_payload, destination):
    passing_times = json_payload['points'][0]['passingTimes']
    str_expected_arrival_times = [passing_time['expectedArrivalTime'] for passing_time in passing_times if passing_time['destination']['fr']==destination]
    print(str_expected_arrival_times)
    expected_arrival_times = [datetime.fromisoformat(str_expected_arrival_time) for str_expected_arrival_time in str_expected_arrival_times]
    return expected_arrival_times 

In [5]:
def compute_time_diff(start, end):
    if isinstance(start, datetime_time): # convert to datetime
        assert isinstance(end, datetime_time)
        start, end = [datetime.combine(datetime.min, t) for t in [start, end]]
    if start <= end: # e.g., 10:33:26-11:15:49
        return end - start
    else: # end < start e.g., 23:55:00-00:25:00
        end += timedelta(1) # +day
        assert end > start
        return end - start

In [6]:
from datetime import datetime, time as datetime_time, timedelta
import pytz


class TimeUtils:

    @staticmethod
    def get_current_localized_time():
        current_time_utc = datetime.utcnow()
        return pytz.utc.localize(current_time_utc)

    @staticmethod
    def compute_time_diff(start, end):
        if isinstance(start, datetime_time): # convert to datetime
            assert isinstance(end, datetime_time)
            start, end = [datetime.combine(datetime.min, t) for t in [start, end]]
        if start <= end: # e.g., 10:33:26-11:15:49
            return TimeUtils._convert_timedelta(end - start)
        else: # end < start e.g., 23:55:00-00:25:00
            end += timedelta(1) # +day
            assert end > start
            return TimeUtils._convert_timedelta(end - start)

    @staticmethod
    def _convert_timedelta(duration):
        duration_dict = {}
        days, seconds = duration.days, duration.seconds
        duration_dict['days'] = days
        hours = seconds // 3600
        minutes = (seconds % 3600) // 60
        seconds = (seconds % 60)
        duration_dict['hours'] = hours
        duration_dict['minutes'] = minutes
        duration_dict['seconds'] = seconds

        return duration_dict


In [7]:
from dataclasses import dataclass, field
from typing import List, Optional, Any

from dataclasses_json import dataclass_json, config, LetterCase

from datetime import datetime

@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class Message:
    fr: str = ""
    nl: str = ""
        
@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class Destination:
    fr: str = ""
    nl: str = ""

@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class PassingTime:
    destination: Optional[Destination] = None
    message: Optional[Message] = None
    line_id: str = ""
    expected_arrival_time: str = ""
    arriving_in_dict: dict = field(init=False, repr=False)
        
    def __post_init__(self):
        self.expected_arrival_time = datetime.fromisoformat(self.expected_arrival_time)
        current_localized_time = TimeUtils.get_current_localized_time()
        self.arriving_in_dict = TimeUtils.compute_time_diff(current_localized_time, self.expected_arrival_time)


    def __str__(self):
        return f'{self.arriving_in_dict}'

        

@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class PointPassingTimes:
    passing_times: List[PassingTime] = field(default_factory=list)
    point_id: str = ""



In [34]:
legrand_stop_id = "1047F"
bearer = 'c0b023736cbee78c8c2cd9e34a475013'
response = make_api_request(legrand_stop_id, bearer)
response

{'points': [{'passingTimes': [{'destination': {'fr': 'ROODEBEEK',
      'nl': 'ROODEBEEK'},
     'expectedArrivalTime': '2020-04-02T13:56:00+02:00',
     'lineId': '8'},
    {'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
     'expectedArrivalTime': '2020-04-02T13:59:00+02:00',
     'lineId': '93',
     'message': {'fr': 'NE PAS EMBARQUER', 'nl': 'NIET INSTAPPEN'}},
    {'destination': {'fr': 'ROODEBEEK', 'nl': 'ROODEBEEK'},
     'expectedArrivalTime': '2020-04-02T14:10:00+02:00',
     'lineId': '8'},
    {'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
     'expectedArrivalTime': '2020-04-02T14:13:00+02:00',
     'lineId': '93',
     'message': {'fr': 'NE PAS EMBARQUER', 'nl': 'NIET INSTAPPEN'}}],
   'pointId': '1047'}]}

In [35]:
passages = PointPassingTimes.schema().load(response['points'], many=True) 

passages

[PointPassingTimes(passing_times=[PassingTime(destination=Destination(fr='ROODEBEEK', nl='ROODEBEEK'), message=None, line_id='8', expected_arrival_time=datetime.datetime(2020, 4, 2, 13, 56, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), PassingTime(destination=Destination(fr='LEGRAND', nl='LEGRAND'), message=Message(fr='NE PAS EMBARQUER', nl='NIET INSTAPPEN'), line_id='93', expected_arrival_time=datetime.datetime(2020, 4, 2, 13, 59, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), PassingTime(destination=Destination(fr='ROODEBEEK', nl='ROODEBEEK'), message=None, line_id='8', expected_arrival_time=datetime.datetime(2020, 4, 2, 14, 10, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))), PassingTime(destination=Destination(fr='LEGRAND', nl='LEGRAND'), message=Message(fr='NE PAS EMBARQUER', nl='NIET INSTAPPEN'), line_id='93', expected_arrival_time=datetime.datetime(2020, 4, 2, 14, 13, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))))], point

In [36]:
expected_arrival_times = parse_waiting_times(response, 'ROODEBEEK' )

current_time_utc = datetime.utcnow()
current_time_be = pytz.utc.localize(current_time_utc)
tram_arrival_time = expected_arrival_times[0]

waiting_time = compute_time_diff(current_time_be, tram_arrival_time)
print(waiting_time)


['2020-04-02T13:56:00+02:00', '2020-04-02T14:10:00+02:00']
0:17:16.133689


In [38]:
def get_shapefiles():
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/Files/2.0/Shapefiles" + stop_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()

In [41]:
@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class LinePoint:
    id: str = ""
    order: int = 0


@dataclass_json(letter_case=LetterCase.CAMEL)
@dataclass
class LineInfo:
    destination: Optional[Destination] = None
    direction: str = ""
    line_id: str = ""
    points: List[LinePoint] = field(default_factory=list)

    def __post_init__(self):
        pass

In [39]:
def make_line_info_api_request(line_id, bearer):
    open_data_api_endpoint = "https://opendata-api.stib-mivb.be/NetworkDescription/1.0/PointByLine/" + line_id
    headers = {"Accept": "application/json"}

    response = requests.get(open_data_api_endpoint, auth=BearerAuth(bearer))
    
    return response.json()


In [43]:
line_info_api_response = make_line_info_api_request("93", bearer)
line_info_api_response

{'lines': [{'destination': {'fr': 'LEGRAND', 'nl': 'LEGRAND'},
   'direction': 'Suburb',
   'lineId': '93',
   'points': [{'id': '5400', 'order': 1},
    {'id': '5402', 'order': 2},
    {'id': '5399F', 'order': 3},
    {'id': '5403', 'order': 4},
    {'id': '5010', 'order': 5},
    {'id': '6501', 'order': 6},
    {'id': '6505', 'order': 7},
    {'id': '6504', 'order': 8},
    {'id': '4130F', 'order': 9},
    {'id': '6100', 'order': 10},
    {'id': '6122', 'order': 11},
    {'id': '1073', 'order': 12},
    {'id': '6103F', 'order': 13},
    {'id': '6178', 'order': 14},
    {'id': '5762', 'order': 15},
    {'id': '1646F', 'order': 16},
    {'id': '6012G', 'order': 17},
    {'id': '6412F', 'order': 18},
    {'id': '6014', 'order': 19},
    {'id': '6437F', 'order': 20},
    {'id': '6308F', 'order': 21},
    {'id': '6309F', 'order': 22},
    {'id': '6310F', 'order': 23},
    {'id': '6311', 'order': 24},
    {'id': '6312', 'order': 25},
    {'id': '6313', 'order': 26},
    {'id': '6314', 'ord

In [45]:
lines = LineInfo.schema().load(line_info_api_response['lines'], many=True) 

lines

[LineInfo(destination=Destination(fr='LEGRAND', nl='LEGRAND'), direction='Suburb', line_id='93', points=[LinePoint(id='5400', order=1), LinePoint(id='5402', order=2), LinePoint(id='5399F', order=3), LinePoint(id='5403', order=4), LinePoint(id='5010', order=5), LinePoint(id='6501', order=6), LinePoint(id='6505', order=7), LinePoint(id='6504', order=8), LinePoint(id='4130F', order=9), LinePoint(id='6100', order=10), LinePoint(id='6122', order=11), LinePoint(id='1073', order=12), LinePoint(id='6103F', order=13), LinePoint(id='6178', order=14), LinePoint(id='5762', order=15), LinePoint(id='1646F', order=16), LinePoint(id='6012G', order=17), LinePoint(id='6412F', order=18), LinePoint(id='6014', order=19), LinePoint(id='6437F', order=20), LinePoint(id='6308F', order=21), LinePoint(id='6309F', order=22), LinePoint(id='6310F', order=23), LinePoint(id='6311', order=24), LinePoint(id='6312', order=25), LinePoint(id='6313', order=26), LinePoint(id='6314', order=27), LinePoint(id='5405', order=28)

In [7]:
slots = {"line_id":"93"}
slots

{'line_id': '93'}

In [8]:
slots['line_id']

'93'